In [4]:
import os
from pathlib import Path

os.chdir('../')

In [35]:
import re

In [93]:
PT_VERBS = ['tratar', 'estabelecer', 'marcar', 'vir', 'colocar',\
           'fechar', 'dar', 'cair', 'encontrar', 'registrar',\
           'levar', 'receber', 'apresentar', 'passar', 'deixar',\
           'chegar', 'ficar', 'fazer', 'ter', 'ser']

In [380]:
def list_files(startpath):
    selected_files = []
    for root, dirs, files in os.walk(startpath):
        level = root
        for f in files:
            if 'dls/verb' in level and '.xml' in f and 'ontology' not in f:
                selected_files.append(level + '/' + f)
    return selected_files

In [381]:
selected_files = list_files(Path.cwd() / 'data/CSTNews 6.0')

In [444]:
verbs_file = {}
id_synstes = []
with open(Path.cwd() / 'data/wordnet/index.sense') as f:
    for ix, line in enumerate(f.read().split('\n')):
        if len(line) > 1:
            verbs_file[line.split()[0]] = line.split()[1]
            id_synstes.append(line.split()[1])
f.close()
id_synstes = list(set(id_synstes))

In [453]:
input_sentences = []
stem_sentences = []
output_sentences = []
output_selected_verb_sentences = []
selected_arrays = []
total_anns = 0
for name_file in selected_files:
    with open(name_file, 'r') as f:
        xml = f.read().lower()
        texts = re.findall(r'<texto(.*?)</texto>', xml, re.DOTALL)
        
        for text in texts:
            sentences = re.findall(r'<p>(.*?)</p>', text, re.DOTALL)
            for sentence in sentences:
                tokens = re.findall(r'<token>(.*?)</token>', text, re.DOTALL)
                temp_in = []
                temp_out = []
                temp_stem = []
                temp_sel = []
                temp = []
                for token in tokens:
                    word_value = re.findall(r'<valor>(.*?)</valor>', token, re.DOTALL)
                    sense_value = re.findall(r'<synset selecionado="true">(.*?)</synset>', token, re.DOTALL)
                    stem_value = re.findall(r'<lema>(.*?)</lema>', token, re.DOTALL)
                    
                    temp_in.append(word_value[0])
                    temp_stem.append(stem_value[0])
                    if len(sense_value) > 0:
                        temp.append(word_value[0] + '<' + (8 - len(sense_value[0])) * '0' + sense_value[0] + '>')
                        if stem_value[0] in PT_VERBS:
                            temp_sel.append((8 - len(sense_value[0])) * '0' + sense_value[0])
                        temp_out.append((8 - len(sense_value[0])) * '0' + sense_value[0])
                    else:
                        temp.append(word_value[0])
                        temp_sel.append(word_value[0])
                        temp_out.append(word_value[0])
                        
                selected_arrays.append(' '.join(temp))
                input_sentences.append(' '.join(temp_in))
                output_sentences.append(' '.join(temp_out))
                stem_sentences.append(' '.join(temp_stem))
                output_selected_verb_sentences.append(' '.join(temp_sel))

In [454]:
selected = list(zip(selected_arrays, stem_sentences, output_sentences))
selected = list(set(selected))

In [455]:
len(selected), len(selected_arrays), len(output_selected_verb_sentences)

(140, 1145, 1145)

In [463]:
d_selected_words = {verb:0 for verb in PT_VERBS}
total_anns = 0
not_exist = []
for ix, (line_w, line_s, line_o) in enumerate(selected):
            for word_w, word_s, word_o in zip(line_w.split(), line_s.split(), line_o.split()): 
                if '<' in word_w and '>' in word_w:
                    if re.sub(r'<(.*?)>', '', word_s) in PT_VERBS:
                        d_selected_words[word_s] += 1
                        total_anns += 1
                    if word_o not in id_synstes:
                        not_exist.append(word_o)
print('ocurrencias :', total_anns, len(not_exist))

ocurrencias : 1326 0


In [425]:
d_selected_words

{'apresentar': 39,
 'cair': 24,
 'chegar': 57,
 'colocar': 20,
 'dar': 55,
 'deixar': 51,
 'encontrar': 27,
 'estabelecer': 12,
 'fazer': 119,
 'fechar': 21,
 'ficar': 71,
 'levar': 34,
 'marcar': 17,
 'passar': 47,
 'receber': 37,
 'registrar': 27,
 'ser': 478,
 'ter': 159,
 'tratar': 11,
 'vir': 20}

In [459]:
sentences = list(zip(input_sentences, output_sentences, output_selected_verb_sentences))
sentences = list(set(sentences))
len(sentences)

140

In [467]:
sentences[10]

('\ufeff o presidente luiz inácio lula da silva disse na manhã desta segunda-feira , no programa café com o presidente , que o brasil se transformará em um " canteiro de obra " . o governo federal prepara o anúncio de uma série de investimentos na malha de transportes . eles devem ser anunciados pelo presidente na próxima semana , após viagem pela américa central . os recursos para a malha de transportes fazem parte dos r $ 504 bilhões previstos no programa de aceleração do crescimento ( pac ) . lula ressaltou que algumas das obras a serem anunciadas já estão em andamento e outras devem começar de imediato . também está prevista a inclusão de obras que ainda necessitam de licenciamento . - o dado concreto é que nós vamos fazer deste país um verdadeiro canteiro de obra em se tratando de infra-estrutura - disse o presidente . ele não antecipou o volume de recursos nem onde serão aplicados . mas , segundo o presidente , as obras de infra-estrutura serão realizadas em rodovias , ferrovias 

In [461]:
f_in_all = open(Path.cwd() / 'data/disambiguation/test/all_verbs_in.txt', 'w')
f_out_all = open(Path.cwd() / 'data/disambiguation/test/all_verbs_out.txt', 'w') 
f_in_sel = open(Path.cwd() / 'data/disambiguation/test/verbs_selected_in.txt', 'w')
f_out_sel = open(Path.cwd() / 'data/disambiguation/test/verbs_selected_out.txt', 'w')

for input_s, output_s_all, output_s_sel in sentences:
    #print(input_s)
    input_s = ' '.join(input_s.split()[1:])
    output_s_all = ' '.join(output_s_all.split()[1:])
    output_s_sel = ' '.join(output_s_sel.split()[1:])
    
    f_in_all.write(input_s + '\n')
    f_out_all.write(output_s_all + '\n')
    
    f_in_sel.write(input_s + '\n')
    f_out_sel.write(output_s_sel + '\n')
    
f_in_all.close()
f_out_all.close()
f_in_sel.close()
f_out_sel.close()

In [470]:
f_i = open(Path.cwd() / 'data/disambiguation/test/verbs_selected_in.txt', 'r')
f_o = open(Path.cwd() / 'data/disambiguation/test/verbs_selected_out.txt', 'r')
f = open(Path.cwd() / 'data/disambiguation/test/verbs_selected.tsv', 'w')
array_senses = []
for l_i, l_o in zip(f_i.read().split('\n'), f_o.read().split('\n')):
    temp = []
    for t_i, t_o in zip(l_i.split('.'), l_o.split('.')):
        t_i
        f.write(t_i + '\t' + t_o + '\n')
f.close()
f_i.close()
f_o.close()

In [471]:
f_i = open(Path.cwd() / 'data/disambiguation/test/all_verbs_in.txt', 'r')
f_o = open(Path.cwd() / 'data/disambiguation/test/all_verbs_out.txt', 'r')
f = open(Path.cwd() / 'data/disambiguation/test/all_verbs.tsv', 'w')
for l_i, l_o in zip(f_i.read().split('\n'), f_o.read().split('\n')):
    for t_i, t_o in zip(l_i.split('.'), l_o.split('.')):
        f.write(t_i + '\t' + t_o + '\n')
f.close()
f_i.close()
f_o.close()